In [1]:
import sys
import os

os.chdir("d:/MyProjects/Project-01/Text-Summarizer-Project")

# Add the src directory to PYTHONPATH
sys.path.append(os.path.abspath("src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,  # Ensure this is valid
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
        )
        return model_trainer_config

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from datasets import load_from_disk
import torch

c:\Users\Juges Debnath\anaconda3\envs\myvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-19 21:44:43,294: INFO: config: 54: PyTorch version 2.6.0 available.]


In [6]:

# from transformers import Seq2SeqTrainingArguments 
# from transformers import Seq2SeqTrainer  
# from datasets import load_dataset, load_from_disk
# import torch

In [7]:
# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config

#     def train(self):
#         device = "cuda" if torch.cuda.is_available() else "cpu"    
#         tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
#         model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
#         seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

#         # Load tokenized dataset from disk
#         dataset_samsum_pt = load_from_disk(str(self.config.data_path))

#         trainer_args = Seq2SeqTrainingArguments(
#             output_dir=self.config.root_dir,
#             num_train_epochs=self.config.num_train_epochs,
#             warmup_steps=self.config.warmup_steps,
#             per_device_train_batch_size=self.config.per_device_train_batch_size,
#             weight_decay=self.config.weight_decay,
#             logging_steps=self.config.logging_steps,
#             evaluation_strategy=self.config.evaluation_strategy,
#             eval_steps=self.config.eval_steps,
#             save_steps=self.config.save_steps,
#             gradient_accumulation_steps=self.config.gradient_accumulation_steps,
#             predict_with_generate=True,  # ✅ Important for summarization
#             fp16=torch.cuda.is_available(),  # ✅ Enables mixed precision on GPU
#         )

#         trainer = Seq2SeqTrainer(  # ✅ Use Seq2SeqTrainer instead of Trainer
#             model=model_pegasus,
#             args=trainer_args,
#             tokenizer=tokenizer,  # ✅ Needed for decoding predictions
#             data_collator=seq2seq_data_collator,
#             train_dataset=dataset_samsum_pt["train"],  # ✅ Train split (not test)
#             eval_dataset=dataset_samsum_pt["validation"],
#         )

#         trainer.train()

#         # Save the fine-tuned model and tokenizer
#         model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
#         tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"    
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Load the dataset
        dataset_samsum_pt = load_from_disk(str(self.config.data_path))

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"],
        )
        trainer.train()

        # Save the fine-tuned model
        model_save_path = model_pegasus.save_pretrained(os.path.join(self.config.root_dir,
                                                                      "pegasus-samsum-model"))

        # Save the tokenizer
        tokenizer_save_path = tokenizer.save_pretrained(os.path.join(self.config.root_dir, 
                                                                     "tokenizer"))

In [9]:
# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config

#     def train(self):
#         device = "cuda" if torch.cuda.is_available() else "cpu"    
#         tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
#         model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
#         seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
#         dataset_samsum_pt = load_from_disk(str(self.config.data_path))

#         trainer_args = Seq2SeqTrainingArguments(
#             output_dir=self.config.root_dir,
#             num_train_epochs=self.config.num_train_epochs,
#             warmup_steps=self.config.warmup_steps,
#             per_device_train_batch_size=self.config.per_device_train_batch_size,
#             weight_decay=self.config.weight_decay,
#             logging_steps=self.config.logging_steps,
#             evaluation_strategy=self.config.evaluation_strategy,
#             eval_steps=self.config.eval_steps,
#             save_steps=self.config.save_steps,
#             gradient_accumulation_steps=self.config.gradient_accumulation_steps,
#             predict_with_generate=True,
#             fp16=torch.cuda.is_available(),
#         )

#         trainer = Seq2SeqTrainer(
#             model=model_pegasus,
#             args=trainer_args,
#             tokenizer=tokenizer,
#             data_collator=seq2seq_data_collator,
#             train_dataset=dataset_samsum_pt["train"],
#             eval_dataset=dataset_samsum_pt["validation"],
#         )

#         trainer.train()

#         model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
#         tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [10]:
try:
    # Create a ConfigurationManager object
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-19 21:44:44,099: INFO: common: 39: Directory: artifacts created successfully]
[2025-04-19 21:44:44,101: INFO: common: 39: Directory: artifacts/model_trainer created successfully]


c:\Users\Juges Debnath\anaconda3\envs\myvenv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: Directory artifacts/data_transformation not found